# Transformação de perspectiva de 4 pontos

Importando bibliotecas

In [1]:
import numpy as np
import cv2

1. Função para ordenar os pontos

In [2]:
def order_points(pts):
    # inicializar a lista de coordenadas que serão ordenadas
    # ordem: superior-esquerdo, superior-direito, inferior-direito, inferior-esquerdo
    # np.zeros => ((nº linhas, nº colunas), tipo_variavel)
    rect = np.zeros((4,2), dtype='float32')
    
    # o ponto superior esquerdo terá a menor soma, enquanto
    # o ponto inferior direito terá a maior soma
    s = pts.sum(axis=1) # soma por linha
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # calcula a diferença entre os pontos
    # o canto superior direito terá uma diferença menor
    # o canto inferior esquerdo terá uma diferença maior
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    # retorna as coordenadas ordenadas
    return rect

2. Função de transformação

In [3]:
def four_point_tranform(image, pts):
    # obtem os pontos ordenados no formato correto
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    # calcula a maior largura da imagem com base nos pontos (coordenada X)
    # distância entre o br e bl
    # distância entre o tr e tl
    # calcula a maior distância
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    
    # calcula a maior altura da imagem com base nos pontos (coordenada Y)
    # distância entre tr e br
    # distância entre tl e bl
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # agora que temos as dimenções da nova imagem
    # construimos os pontos para obter a vista de 'cima' (vista desejada)
    # seguindo a ordem dos pontos: tl, tr, br, bl
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = 'float32')
    
    # calcula a matriz de transformação de perspectiva
    M = cv2.getPerspectiveTransform(rect, dst)
    # aplica a matriz de transformação de perspectiva
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    # retorna a imagem distorcida
    return warped

3. Utilizando 

Os pontos de coordenadas foram inseridos manualmente, apenas para aprendizado de como funciona as funções

- images/example_01.png --coords "[(73, 239), (356, 117), (475, 265), (187, 443)]"
- images/example_02.png --coords "[(101, 185), (393, 151), (479, 323), (187, 441)]"
- images/example_03.png --coords "[(63, 242), (291, 110), (361, 252), (78, 386)]"

In [4]:
# Carregando  localização das imagens e coordenadas
image_1 = "./images/example_01.png"
pts_1 = [(73, 239), (356, 117), (475, 265), (187, 443)]

image_2 = "./images/example_02.png"
pts_2 = [(101, 185), (393, 151), (479, 323), (187, 441)]

image_3 = "./images/example_03.png"
pts_3 = [(63, 242), (291, 110), (361, 252), (78, 386)]

In [5]:
# carregando imagem e coordenadas
image=cv2.imread(image_1)
coords= np.array([(73, 239), (356, 117), (475, 265), (187, 443)])

image_2=cv2.imread(image_2)
coords_2= np.array([(101, 185), (393, 151), (479, 323), (187, 441)])

image_3=cv2.imread(image_3)
coords_3= np.array([(63, 242), (291, 110), (361, 252), (78, 386)])

# aplicando transformação
warped = four_point_tranform(image, coords)
warped_2 = four_point_tranform(image_2, coords_2)
warped_3 = four_point_tranform(image_3, coords_3)

In [6]:
# visualizando resultado
cv2.imshow("Original", image)
cv2.imshow("Warped", warped)
cv2.waitKey(0)

48

In [7]:
# visualizando resultado
cv2.imshow("Original", image_2)
cv2.imshow("Warped", warped_2)
cv2.waitKey(0)

48

In [ ]:
# visualizando resultado
cv2.imshow("Original", image_3)
cv2.imshow("Warped", warped_3)
cv2.waitKey(0)